# Velib stations usage

Inspired from notebooks: https://nbviewer.jupyter.org/github/jorisvandenbossche/talks/blob/master/2017_EuroScipy_geopandas/geopandas_demo.ipynb

In [ ]:
%matplotlib inline
import os

import pandas as pd
import geopandas as gpd
import folium

pd.options.display.max_rows = 10

In [ ]:
# Parameters
year=2019
month=11
day=7
hour=12

## Get stations dataset

Copy geojson dataset from smovengo minio bucket

In [ ]:
# Get path to data
data_path = os.path.join("s3://smovengo/", f"{year:04d}", f"{month:02d}", f"{day:02d}", f"{hour:02d}.geojson")

In [ ]:
%%bash -s $data_path
aws s3 cp --endpoint-url $AWS_S3_ENDPOINT $1 stations.geojson 

# Get district and velib stations

From opendata.paris.fr

- https://opendata.paris.fr/explore/dataset/quartier_paris
- https://opendata.paris.fr/explore/dataset/velib-disponibilite-en-temps-reel

## District dataset and overview

In [ ]:
#https://jorisvandenbossche.github.io/blog/2017/09/19/geopandas-cython/
#https://opendata.paris.fr/explore/dataset/quartier_paris/export/?location=12,48.85889,2.34692&basemap=jawg.streets
#https://opendata.paris.fr/explore/dataset/velib-disponibilite-en-temps-reel/export/
districts = gpd.read_file("https://opendata.paris.fr/explore/dataset/quartier_paris/download/?format=geojson&timezone=Europe/Paris")
districts.head()

In [ ]:
districts.plot(figsize=(10, 10))

## Stations dataset and overview

In [ ]:
#"https://opendata.paris.fr/explore/dataset/velib-disponibilite-en-temps-reel/download/?format=geojson&timezone=Europe/Paris"
stations = gpd.read_file("stations.geojson")
stations[['station_state', 'station_name', 'nbedock', 'nbebike', 'nbfreeedock', 'nbebikeoverflow', 'geometry']].head()

In [ ]:
operative_stations = stations[stations['station_state'] == 'Operative'].copy()
operative_stations['nbedock'].hist()

## Aggregate velib per district

In [ ]:
# Counts available velib per districts
stations_per_district = gpd.sjoin(stations, districts[['l_qu', 'geometry']].copy(), op='within')
counts = stations_per_district.groupby('l_qu').size()
velib_per_districts = districts.merge(counts.reset_index(name='nb_ebike_stations'))
velib_per_districts['nb_ebike_stations_relative_to_area'] = velib_per_districts['nb_ebike_stations'] / velib_per_districts.geometry.area

In [ ]:
ax = velib_per_districts.plot(column='nb_ebike_stations', cmap='inferno', figsize=(15, 6), legend=True)
ax.set_axis_off()